<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_14_Notebook_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **LangChain [Dados Estruturados]**
---



In [ ]:
#@title Instalando o pacote LangChain
!pip install langchain -q U

In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

0.3.27


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.


In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API da OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


## **Dados Estruturados**

**Dados estruturados** são dados que são organizados e formatados de maneira sistemática, permitindo fácil acesso, análise e manipulação.

Frequentemente é útil ter um modelo que retorne uma saída que corresponda a um esquema específico.

**Método `.with_structured_output()`**

Esta é a maneira mais fácil e confiável de obter saídas estruturadas. O método `with_structured_output()` é implementado para modelos que fornecem APIs nativas para estruturar saídas, como chamadas de ferramentas/funções ou modo JSON, e utiliza essas capacidades internamente.

>

Este método recebe um esquema como entrada, que especifica os nomes, tipos e descrições dos atributos de saída desejados. O método retorna um `Runnable` semelhante a um modelo, exceto que, em vez de produzir strings ou Mensagens, ele produz objetos correspondentes ao esquema fornecido. O esquema pode ser especificado como uma classe `TypedDict`, um `JSON Schema` ou uma classe `Pydantic`. Se `TypedDict` ou `JSON Schema` forem usados, então um dicionário será retornado pelo `Runnable`, e se uma classe `Pydantic` for usada, então um objeto `Pydantic` será retornado.

>

Como exemplo, vamos fazer com que um modelo gere uma piada e separe a preparação do desfecho da piada:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4o-mini", api_key = OPENAI_API_KEY)

**Classe `Pydantic`**

Se quisermos que o modelo retorne um objeto `Pydantic`, basta passar a classe `Pydantic` desejada. A principal vantagem de usar `Pydantic` é que a saída gerada pelo modelo será validada. O `Pydantic` levantará um erro se algum campo obrigatório estiver faltando ou se algum campo for do tipo errado.

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field

# Pydantic
class Piada(BaseModel):
    """Piada para contar ao usuário."""

    introducao: str = Field(description="A introdução da piada")
    arremate: str = Field(description="O desfecho da piada")
    avaliacao: Optional[int] = Field(
        default=None, description="Quão engraçada é a piada, de 1 a 10"
    )

In [ ]:
modelo_estruturado = modelo.with_structured_output(Piada)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

Piada(introducao='Por que os gatos não conseguem jogar cartões?', arremate='Porque eles sempre ficam em cima da mesa!', avaliacao=7)

Além da estrutura da classe `Pydantic`, o nome da classe, a *docstring* e os nomes e as descrições fornecidas dos parâmetros são muito importantes. Na maioria das vezes, `with_structured_output` está usando a API de chamada de funções/ferramentas de um modelo, e você pode pensar efetivamente em todas essas informações como sendo adicionadas ao *prompt* do modelo.

**`TypedDict` ou `JSON Schema`**

Se você não quiser usar `Pydantic`, explicitamente não quiser validação dos argumentos ou quiser ser capaz de transmitir as saídas do modelo, pode definir seu esquema usando uma classe `TypedDict`. Opcionalmente, podemos usar uma sintaxe especial `Annotated` suportada pelo **LangChain**, que permite especificar o valor padrão e a descrição de um campo. Observe que o valor padrão não é preenchido automaticamente se o modelo não o gerar; ele é usado apenas na definição do esquema que é passado para o modelo.

In [ ]:
from typing_extensions import Annotated, TypedDict

# TypedDict
class Piada(TypedDict):
    """Piada para contar ao usuário."""

    introducao: Annotated[str, ..., "introducao"]
    arremate: Annotated[str, ..., "O desfecho da piada"]
    avaliacao: Annotated[Optional[int], None, "Quão engraçada é a piada, de 1 a 10"]

In [ ]:
modelo_estruturado = modelo.with_structured_output(Piada)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

{'introducao': 'Por que os gatos não gostam de computadores?',
 'arremate': 'Porque eles ficam aterrorizados com o mouse!',
 'avaliacao': 7}

De maneira equivalente, podemos passar um dicionário `JSON Schema`. Isso não requer importações ou classes e torna muito claro como cada parâmetro é documentado, com o custo de ser um pouco mais verboso.

In [ ]:
json_schema = {
    "title": "piada",
    "description": "Piada para contar ao usuário.",
    "type": "object",
    "properties": {
        "introducao": {
            "type": "string",
            "description": "A introdução da piada",
        },
        "arremate": {
            "type": "string",
            "description": "O desfecho da piada",
        },
        "avaliacao": {
            "type": "integer",
            "description": "Quão engraçada é a piada, de 1 a 10",
            "default": None,
        },
    },
    "required": ["introducao", "arremate"],
}


In [ ]:
modelo_estruturado = modelo.with_structured_output(json_schema)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

{'introducao': 'Por que os gatos não brincam de esconde-esconde?',
 'arremate': 'Porque eles sempre se escondem na caixa!',
 'avaliacao': 7}

**Escolhendo entre múltiplos esquemas**

A maneira mais simples de permitir que o modelo escolha entre vários esquemas é criar um esquema "pai" que tenha um atributo do tipo `Union`:

In [ ]:
from typing import Union

# Pydantic

class Piada(BaseModel):
    """Piada para contar ao usuário."""

    introducao: str = Field(description="A introdução da piada")
    arremate: str = Field(description="O desfecho da piada")
    avaliacao: Optional[int] = Field(
        default=None, description="Quão engraçada é a piada, de 1 a 10"
    )

class RespostaConversacional(BaseModel):
    """Responda de maneira conversacional. Seja gentil e prestativo."""

    resposta: str = Field(description="Uma resposta conversacional à pergunta do usuário")

class Resposta(BaseModel):
    saida: Union[Piada, RespostaConversacional]

In [ ]:
modelo_estruturado = modelo.with_structured_output(Resposta)

resposta = modelo_estruturado.invoke("Me conte uma piada sobre gatos")

In [ ]:
resposta

Resposta(saida=Piada(introducao='Por que os gatos sempre ganham em videogames?', arremate='Porque eles têm nove vidas!', avaliacao=7))

In [ ]:
resposta.saida

Piada(introducao='Por que os gatos sempre ganham em videogames?', arremate='Porque eles têm nove vidas!', avaliacao=7)

In [ ]:
resposta.saida.introducao

'Por que os gatos sempre ganham em videogames?'

In [ ]:
modelo_estruturado.invoke("Como você está hoje?")

Resposta(saida=RespostaConversacional(resposta='Estou ótimo, obrigado por perguntar! E você, como está hoje?'))

***Few-shot prompting***

***Few-shot prompting*** é uma técnica onde se fornecem alguns exemplos específicos no *prompt* para guiar o modelo na geração de respostas. Em vez de treinar o modelo com uma nova base de dados completa, você apenas inclui alguns exemplos relevantes diretamente no *prompt*.

Para esquemas mais complexos, é muito útil adicionar exemplos ao *prompt*.

A maneira mais simples e universal é adicionar exemplos a uma mensagem do sistema no *prompt*:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sistema = """Você é um comediante hilário. Sua especialidade são piadas do tipo "knock-knock".
Retorne uma piada que tenha a introdução (a resposta para "Quem está aí?") e o arremate final (a resposta para "<introdução> quem?").

Aqui estão alguns exemplos de piadas:

exemplo_usuario: Me conte uma piada sobre gatos
exemplo_assistente: {{"introducao": "Por que os gatos não jogam cartas?", "arremate": "Porque eles têm medo dos cães que podem trapacear!", "avaliacao": 8}}

exemplo_usuario: Conte outra piada sobre cães
exemplo_assistente: {{"introducao": "Por que os cães levam uma bola para a escola?", "arremate": "Porque eles querem aprender a jogar!", "ratavaliacaoing": 7}}

exemplo_usuario: Agora sobre peixes
exemplo_assistente: {{"introducao": "Por que o peixe foi ao banco?", "arremate": "Para abrir uma conta corrente!", "avaliacao": 9}}"""

# template de prompt
prompt = ChatPromptTemplate.from_messages([("system", sistema), ("human", "{input}")])

# uso do prompt com um modelo estruturado
modelo_estruturado_few_shot = prompt | modelo_estruturado

modelo_estruturado_few_shot.invoke("Me conte uma piada sobre elefantes")

Resposta(saida=Piada(introducao='Por que os elefantes nunca usam computador?', arremate='Porque eles têm medo do mouse!', avaliacao=7))

Outro exemplo:

In [ ]:
# Pydantic

class Produto(BaseModel):
    """Descrição de um produto."""

    nome: str = Field(description="O nome do produto")
    descricao: str = Field(description="A descrição do produto")

In [ ]:
modelo_estruturado = modelo.with_structured_output(Produto)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sistema = """Você é um redator de descrições de produtos experiente. Sua tarefa é criar descrições detalhadas e envolventes para produtos.
Cada descrição deve incluir informações sobre as principais características e benefícios do produto.

Aqui estão alguns exemplos de descrições:

exemplo_usuario: Descreva um smartphone moderno
exemplo_assistente: {{"nome": "Smartphone UltraX", "descricao": "O Smartphone UltraX é equipado com uma tela AMOLED de 6,7 polegadas, processador octa-core de última geração e câmera tripla de 64MP. Oferece um desempenho rápido e uma experiência de usuário excepcional com bateria de longa duração e carregamento rápido."}}

exemplo_usuario: Descreva um aspirador de pó robô
exemplo_assistente: {{"nome": "Aspirador Robô CleanPro", "descricao": "O Aspirador Robô CleanPro é ideal para manter sua casa limpa sem esforço. Com tecnologia de navegação inteligente, ele limpa todos os cantos e vem com um sistema de filtragem HEPA que captura alérgenos e poeira. Compacto e silencioso, é fácil de usar e programar."}}

exemplo_usuario: Agora sobre uma cafeteira
exemplo_assistente: {{"nome": "Cafeteira Espresso Elite", "descricao": "A Cafeteira Espresso Elite oferece uma experiência de café de qualidade barista no conforto de sua casa. Com um sistema de pressão de 15 bar, ela extrai o máximo sabor dos grãos. Possui um moinho integrado e controle de temperatura preciso para preparar o café perfeito a cada vez."}}"""

# template de prompt
prompt = ChatPromptTemplate.from_messages([("system", sistema), ("human", "{input}")])

# uso do prompt com um modelo estruturado
modelo_estruturado_few_shot = prompt | modelo_estruturado

modelo_estruturado_few_shot.invoke("Descreva um tablet moderno")

Produto(nome='Tablet ProTech 12', descricao='O Tablet ProTech 12 é a combinação perfeita de potência e portabilidade. Com uma tela Retina de 12,9 polegadas, suas cores vibrantes e detalhes nítidos tornam a visualização de vídeos e trabalho em aplicativos uma experiência envolvente. Equipado com um processador Snapdragon de última geração, ele oferece desempenho rápido em multitarefas e jogos exigentes. A bateria de longa duração garante que você possa trabalhar ou se divertir ao longo do dia sem se preocupar em recarregar. Além disso, a caneta stylus incluída permite uma escrita e desenho mais precisos, ideal para profissionais criativos.')

Para modelos que suportam mais de um meio de estruturar saídas, você pode especificar qual método usar com o argumento `method=`.

In [ ]:
modelo_estruturado = modelo.with_structured_output(None, method="json_mode")

modelo_estruturado.invoke(
    "Descreva um tablet moderno, responda em JSON com as chaves `nome` e `descricao`"
)

{'nome': 'Tablet X Pro 2023',
 'descricao': 'O Tablet X Pro 2023 é um dispositivo premium com tela de 12,9 polegadas Retina Liquid, oferecendo cores vibrantes e excelente nitidez. Equipado com um processador ultra-rápido, 8 GB de RAM e opções de armazenamento de até 512 GB, ele é perfeito para multitarefa e desempenho em aplicativos gráficos. Possui suporte para caneta stylus, permitindo desenho e anotações precisas, e uma bateria que dura até 15 horas. Conectividade Wi-Fi 6 e 5G garantem velocidade de internet, enquanto um design leve e elegante com molduras finas proporciona portabilidade e estilo. O tablet também vem com sistema de som surround estéreo e câmeras de alta resolução para fotos e videochamadas.'}

Modelos de linguagem não são perfeitos ao gerar saídas estruturadas, especialmente quando os esquemas se tornam complexos. Você pode evitar a geração de exceções e lidar com a saída bruta você mesmo passando `include_raw=True`. Isso altera o formato da saída para conter a mensagem bruta, o valor analisado (se bem-sucedido) e quaisquer erros resultantes.

In [ ]:
modelo_estruturado = modelo.with_structured_output(Produto, include_raw=True)

modelo_estruturado.invoke("Descreva um tablet moderno")

{'raw': AIMessage(content='{"nome":"Tablet Ultra Slim 2023","descricao":"O Tablet Ultra Slim 2023 é um dispositivo leve e portátil, com um design elegante e moderno. Possui uma tela de 12,9 polegadas com tecnologia AMOLED, oferecendo cores vibrantes e alta definição, ideal para assistir filmes e jogar. Equipado com um processador de última geração e 8 GB de RAM, garante desempenho fluido em todas as tarefas, desde trabalhos de escritório até jogos pesados. A bateria de longa duração suporta até 12 horas de uso contínuo, e a conectividade inclui Wi-Fi 6, Bluetooth 5.1 e suporte a redes 5G. Com sistema operacional Android 13, o tablet oferece uma vasta gama de aplicativos e uma interface intuitiva. Além disso, sua câmera traseira de 12 MP e frontal de 8 MP permitem tirar fotos e realizar chamadas de vídeo com qualidade excepcional."}', additional_kwargs={'parsed': Produto(nome='Tablet Ultra Slim 2023', descricao='O Tablet Ultra Slim 2023 é um dispositivo leve e portátil, com um design el